In [27]:
#imports

!pip install ipython-cache
import cache_magic
from random import randint
import sys
import math

# MDP Implementations

In [28]:
# Creating MDP representation for the grid world and organizing its functionalities

class ActionResult:
    def __init__ (self, resultState, prob, reward):
        self.resultState = resultState
        self.prob = prob
        self.reward = reward
        
    def __str__(self):
        return str(self.resultState) + " " + str(self.prob) + " " + str(self.reward)


class Action:
    # abstraction for an action. Expects a state on which this action
    # can be taken and a list of ActionResult
    def __init__ (self, state, results, label):
        self.state = state
        self.results = results
        self.label = label
    
    def __str__(self):
        return str(self.state) + " " + str([str(r) for r in self.results])
    
class State:
    def __init__(self, id, actions, value):
        self.id = id
        self.actions = actions
        self.value = value       

        def __str__(self):
            return str(self.id) + " " + str(self.value) + " " + str(self.actions)

class DeterministicGridWorldMDP(object):
    # creates a gridworld representation where the reward for every transition
    # is given in the rewards matrix which means the reward the agent gets when achieving that reward
    def __init__ (self, rewards, rows, cols):
        self.transitions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        self.nTransitionsPerState = len(self.transitions)
        self.rows = rows
        self.cols = cols
        self.rewards = rewards
        self.policy = [[randint(0, self.nTransitionsPerState-1) for j in range(cols)] for i in range(rows)]
        self.value = [[0 for j in range(cols)] for i in range(rows)]
        self.states = [[State((i, j), self.generateActions(i, j), 0) for j in range(cols)] for i in range(rows)]
    
    def generateActions(self, i, j):
        actions = []
        prettyDirs = ['r', 'l', 'd', 'u']
        for transition in range(self.nTransitionsPerState):
            resultState = (i + self.transitions[transition][0], j + self.transitions[transition][1])
            if self.isValidTransition(resultState):
                actions.append(Action((i, j), [ActionResult(resultState, 1, self.rewards[resultState[0]][resultState[1]])], prettyDirs[transition]))
         
        return actions
    
    def getReward(self, dest):
        return self.rewards[dest[0]][dest[1]]

    def getStateValue(self, state):
        return self.states[state[0]][state[1]].value

    def isValidTransition(self, dest):
        return  not ((dest[0] < 0 or dest[0] >= self.rows) or (dest[1] < 0 or dest[1] >= self.cols))
    
    def upadateStateValue(self, state):
        maxV = float("-inf")
        actions = self.states[state[0]][state[1]].actions
        for i in range(len(actions)):
            action = actions[i]
            resultState = action.results[0].resultState
            r = action.results[0].reward
            v = self.getStateValue(resultState)
            if r + v > maxV:
                maxV = r + v
                self.policy[state[0]][state[1]] = action.label
                self.states[state[0]][state[1]].value = maxV
    
    def printValues(self):
        for r in self.value:
            print(r)
            
    def getPolicy(self):
        return self.policy
        
        
    def printPolicy(self):
        for r in self.policy:
            print(r)

    def iterateValues(self):
        for i in range(self.rows):
            for j in range(self.cols):
                self.upadateStateValue((i, j))

    def iterateValuesUntilConverge(self, printSteps=False):
        while True:
            values = self.value
            self.iterateValues()
            if printSteps:
                print()
                self.printPolicy()      
            if values == self.value:
                break
                
    def repeatIterateValues(self, n, printSteps=False):
        for i in range(n):
            self.iterateValues()
            if printSteps:
                print()
                self.printPolicy()

# Auxiliary functions

In [29]:
def printMatrix(matrix):
    for r in matrix:
        print(r)
    print()

def generateRandomRewards(row, cols, goalReward, notGoalReward):
    goalCoordX, goalCoordY = randint(0, rows - 1), randint(0, cols - 1)
    rewards = [[notGoalReward for j in range(cols)] for i in range(rows)]
    rewards[goalCoordX][goalCoordY] = goalReward
    return rewards, (goalCoordX, goalCoordY)

def generateRandomRewardsWithObstacles(row, cols, goalReward, notGoalReward, obstacleReward, nObstacles):
    
    rewards = [[notGoalReward for j in range(cols)] for i in range(rows)]
    
    goalCoordX, goalCoordY = randint(0, rows - 1), randint(0, cols - 1)
    goal = (goalCoordX, goalCoordY)
    rewards[goalCoordX][goalCoordY] = goalReward
    
    obstacles = []
    while nObstacles > 0:
        obs = randint(0, rows - 1), randint(0, cols - 1)
        if not obs in obstacles and not obs == goal:
            obstacles.append(obs)
            nObstacles = nObstacles - 1
            rewards[obs[0]][obs[1]] = obstacleReward
    
    return rewards, goal, obstacles


def markGoalAndObstaclesOnGrid(grid, goal, obstacles):
    grid[goal[0]][goal[1]] = 'G'
    for o in obstacles:
        grid[o[0]][o[1]] = 'X'
    
    return grid


# Tests with deterministics gridworld environments

In [30]:
# First dummy smoke test to make sure everything is not absurdly wrong

rewards = [
    [-1, -1, -1],
    [-1, 100, -1],
    [-1, -1, -1]
]

rows, cols = 3, 3

mdp = DeterministicGridWorldMDP(rewards, rows, cols)
mdp.printPolicy()
mdp.repeatIterateValues(10)
print ()
printMatrix(markGoalAndObstaclesOnGrid(mdp.getPolicy(), (1,1), []))

[2, 0, 0]
[3, 3, 1]
[3, 0, 3]

['r', 'd', 'l']
['r', 'G', 'l']
['r', 'u', 'l']



In [31]:
# Testing random test case for 10 rows and cols  

rows, cols = 10, 10

rewards, goal = generateRandomRewards(rows, cols, 100, 0)
mdp = DeterministicGridWorldMDP(rewards, rows, cols)
mdp.printPolicy()
mdp.repeatIterateValues(10)
printMatrix(markGoalAndObstaclesOnGrid(mdp.getPolicy(), goal, []))

[2, 1, 0, 1, 2, 2, 2, 3, 2, 1]
[0, 3, 0, 2, 0, 1, 2, 2, 0, 3]
[3, 2, 1, 3, 0, 3, 3, 2, 1, 2]
[1, 1, 2, 2, 1, 0, 1, 2, 2, 3]
[0, 3, 0, 0, 1, 3, 3, 3, 0, 3]
[2, 0, 0, 3, 0, 1, 1, 1, 0, 2]
[2, 3, 3, 0, 2, 1, 1, 3, 0, 3]
[0, 3, 3, 1, 0, 0, 1, 3, 1, 1]
[3, 2, 1, 0, 1, 0, 3, 1, 2, 1]
[0, 3, 2, 2, 0, 0, 1, 0, 3, 0]
['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'd', 'l']
['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'd', 'l']
['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'd', 'l']
['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'G', 'l']
['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'u', 'l']
['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'u', 'l']
['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'u', 'l']
['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'u', 'l']
['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'u', 'l']
['r', 'r', 'r', 'r', 'r', 'r', 'r', 'r', 'u', 'l']



In [32]:
# Grid world with obstacles (things are getting interesting)

rows, cols = 5, 5

rewards, goal, obs = generateRandomRewardsWithObstacles(rows, cols, 100, 0, -1000, 5)
print ("rewards")
printMatrix(rewards)
print ("goal", goal)
print ("obstacles", obs)
print()

mdp = DeterministicGridWorldMDP(rewards, rows, cols)
mdp.printPolicy()
mdp.repeatIterateValues(10)
print()

printMatrix(markGoalAndObstaclesOnGrid(mdp.getPolicy(), goal, obs))

rewards
[0, 0, 0, 0, -1000]
[0, 0, 0, 0, 100]
[-1000, 0, 0, -1000, 0]
[0, 0, 0, 0, 0]
[0, 0, -1000, -1000, 0]

goal (1, 4)
obstacles [(4, 3), (0, 4), (2, 0), (4, 2), (2, 3)]

[3, 2, 3, 0, 3]
[0, 1, 3, 0, 1]
[3, 0, 0, 1, 1]
[2, 0, 3, 0, 2]
[0, 2, 2, 2, 1]

['r', 'r', 'r', 'd', 'X']
['r', 'r', 'r', 'r', 'G']
['X', 'r', 'u', 'X', 'u']
['r', 'r', 'r', 'r', 'u']
['r', 'u', 'X', 'X', 'u']



In [33]:
# This case proves my iterate until converge is not working propperly

rewards = [
    [0, 0, 0, -1000, 0],
    [0, -1000, 0, 0, -1000],
    [-1000, 0, 0, 0, 0],
    [0, -1000, 100, 0, 0],
    [0, 0, 0, 0, 0]
]

goal = (3, 2)
obs = [(1, 4), (3, 1), (2, 0), (1, 1), (0, 3)]
print ("rewards")
printMatrix(rewards)
print ("goal", goal)
print ("obstacles", obs)
print()

mdp = DeterministicGridWorldMDP(rewards, rows, cols)
mdp.iterateValuesUntilConverge()

printMatrix(markGoalAndObstaclesOnGrid(mdp.getPolicy(), goal, obs))

mdp = DeterministicGridWorldMDP(rewards, rows, cols)
mdp.repeatIterateValues(10)
printMatrix(markGoalAndObstaclesOnGrid(mdp.getPolicy(), goal, obs))


rewards
[0, 0, 0, -1000, 0]
[0, -1000, 0, 0, -1000]
[-1000, 0, 0, 0, 0]
[0, -1000, 100, 0, 0]
[0, 0, 0, 0, 0]

goal (3, 2)
obstacles [(1, 4), (3, 1), (2, 0), (1, 1), (0, 3)]

['r', 'r', 'l', 'X', 'l']
['u', 'X', 'r', 'l', 'X']
['X', 'r', 'd', 'l', 'l']
['d', 'X', 'G', 'l', 'l']
['r', 'r', 'u', 'l', 'l']

['r', 'r', 'd', 'X', 'l']
['u', 'X', 'd', 'l', 'X']
['X', 'r', 'd', 'l', 'l']
['d', 'X', 'G', 'l', 'l']
['r', 'r', 'u', 'l', 'l']



In [34]:
# Another case where iterating until converge do not work

rewards = [
    [0,     0,      0,     0, 0],
    [0,     0,     -1000,  0, -1000],
    [0,     0,     0,      0, 0],
    [0,     -1000, -1000,  -1000, -1000],
    [0,     0,     0,    0, 100]
]

rows, cols = 5, 5

goal = (4, 4)
obs = [(3, 1),(3, 2), (3, 3), (3, 4), (1, 2), (1, 4)]
print ("rewards")
printMatrix(rewards)
print ("goal", goal)
print ("obstacles", obs)
print()

mdp = DeterministicGridWorldMDP(rewards, rows, cols)
mdp.iterateValuesUntilConverge()
printMatrix(markGoalAndObstaclesOnGrid(mdp.getPolicy(), goal, obs))

mdp = DeterministicGridWorldMDP(rewards, rows, cols)
mdp.repeatIterateValues(10)
printMatrix(markGoalAndObstaclesOnGrid(mdp.getPolicy(), goal, obs))

rewards
[0, 0, 0, 0, 0]
[0, 0, -1000, 0, -1000]
[0, 0, 0, 0, 0]
[0, -1000, -1000, -1000, -1000]
[0, 0, 0, 0, 100]

goal (4, 4)
obstacles [(3, 1), (3, 2), (3, 3), (3, 4), (1, 2), (1, 4)]

['r', 'r', 'r', 'r', 'l']
['r', 'l', 'X', 'd', 'X']
['r', 'r', 'r', 'r', 'l']
['d', 'X', 'X', 'X', 'X']
['r', 'r', 'r', 'r', 'G']

['d', 'l', 'l', 'l', 'l']
['d', 'l', 'X', 'd', 'X']
['d', 'l', 'l', 'l', 'l']
['d', 'X', 'X', 'X', 'X']
['r', 'r', 'r', 'r', 'G']



In [35]:
# Lets have variable obstacles with variable penalties

rewards = [
    [0,     0,      0,     0, 0],
    [0,     0,     -1000,  0, -1000],
    [0,     0,     0,      0, 0],
    [0,     -250, -500,  -750, -1000],
    [0,     0,     0,    0, 100]
]

rows, cols = 5, 5

goal = (4, 4)
obs = [(3, 1),(3, 2), (3, 3), (3, 4), (1, 2), (1, 4)]
print ("rewards")
printMatrix(rewards)
print ("goal", goal)
print ("obstacles", obs)
print()

mdp = DeterministicGridWorldMDP(rewards, rows, cols)
mdp.iterateValuesUntilConverge()
printMatrix(markGoalAndObstaclesOnGrid(mdp.getPolicy(), goal, obs))

mdp = DeterministicGridWorldMDP(rewards, rows, cols)
mdp.repeatIterateValues(10)
printMatrix(markGoalAndObstaclesOnGrid(mdp.getPolicy(), goal, obs))

rewards
[0, 0, 0, 0, 0]
[0, 0, -1000, 0, -1000]
[0, 0, 0, 0, 0]
[0, -250, -500, -750, -1000]
[0, 0, 0, 0, 100]

goal (4, 4)
obstacles [(3, 1), (3, 2), (3, 3), (3, 4), (1, 2), (1, 4)]

['r', 'r', 'r', 'r', 'l']
['r', 'l', 'X', 'd', 'X']
['r', 'r', 'r', 'r', 'l']
['d', 'X', 'X', 'X', 'X']
['r', 'r', 'r', 'r', 'G']

['d', 'l', 'l', 'l', 'l']
['d', 'l', 'X', 'd', 'X']
['d', 'l', 'l', 'l', 'l']
['d', 'X', 'X', 'X', 'X']
['r', 'r', 'r', 'r', 'G']



In [36]:
# Lets now give negative reward for moving

rewards = [
    [-100,     -100,      -100,     -100, -10],
    [-100,     -100,     -1000,  -100, -1000],
    [-100,     -100,     -100,      -100, -10],
    [-100,     -250, -500,  -750, -1000],
    [-100,     -100,     -100,    -100, 100]
]

rows, cols = 5, 5

goal = (4, 4)
obs = [(3, 1),(3, 2), (3, 3), (3, 4), (1, 2), (1, 4)]
print ("rewards")
printMatrix(rewards)
print ("goal", goal)
print ("obstacles", obs)
print()

mdp = DeterministicGridWorldMDP(rewards, rows, cols)
mdp.iterateValuesUntilConverge()
printMatrix(markGoalAndObstaclesOnGrid(mdp.getPolicy(), goal, obs))

mdp = DeterministicGridWorldMDP(rewards, rows, cols)
mdp.repeatIterateValues(20)
printMatrix(markGoalAndObstaclesOnGrid(mdp.getPolicy(), goal, obs))

rewards
[-100, -100, -100, -100, -10]
[-100, -100, -1000, -100, -1000]
[-100, -100, -100, -100, -10]
[-100, -250, -500, -750, -1000]
[-100, -100, -100, -100, 100]

goal (4, 4)
obstacles [(3, 1), (3, 2), (3, 3), (3, 4), (1, 2), (1, 4)]

['r', 'r', 'r', 'r', 'l']
['r', 'd', 'X', 'd', 'X']
['r', 'r', 'r', 'r', 'l']
['d', 'X', 'X', 'X', 'X']
['r', 'r', 'r', 'r', 'G']

['d', 'd', 'l', 'l', 'l']
['d', 'd', 'X', 'd', 'X']
['d', 'd', 'l', 'l', 'l']
['d', 'X', 'X', 'X', 'X']
['r', 'r', 'r', 'r', 'G']



In [37]:
# how long it takes to converge

rows, cols = 100, 100

rewards, goal, obs = generateRandomRewardsWithObstacles(rows, cols, 100, 0, -1000, 100)
print ("rewards")
# printMatrix(rewards)
print ("goal", goal)
print ("obstacles", obs)
print()

mdp = DeterministicGridWorldMDP(rewards, rows, cols)
mdp.iterateValuesUntilConverge()
policy1 = mdp.getPolicy()
# printMatrix(markGoalAndObstaclesOnGrid(mdp.getPolicy(), goal, obs))

mdp = DeterministicGridWorldMDP(rewards, rows, cols)
mdp.repeatIterateValues(10)
policy2 = mdp.getPolicy()
# printMatrix(markGoalAndObstaclesOnGrid(mdp.getPolicy(), goal, obs))

mdp = DeterministicGridWorldMDP(rewards, rows, cols)
mdp.repeatIterateValues(50)
policy3 = mdp.getPolicy()

mdp = DeterministicGridWorldMDP(rewards, rows, cols)
mdp.repeatIterateValues(100)
policy4 = mdp.getPolicy()

print(policy1 == policy2)
print(policy2 == policy3)
print(policy3 == policy4)


rewards
goal (68, 6)
obstacles [(21, 56), (74, 42), (50, 94), (52, 63), (68, 88), (85, 73), (21, 96), (45, 17), (47, 64), (67, 83), (47, 34), (28, 86), (74, 29), (76, 85), (68, 34), (72, 28), (54, 48), (27, 22), (55, 51), (53, 68), (48, 78), (46, 90), (53, 98), (29, 59), (56, 22), (84, 88), (45, 55), (33, 56), (82, 27), (77, 67), (65, 30), (60, 62), (64, 93), (23, 5), (1, 95), (12, 56), (39, 53), (71, 48), (53, 69), (23, 74), (67, 70), (54, 80), (30, 34), (19, 23), (51, 38), (2, 5), (50, 74), (53, 85), (39, 51), (16, 11), (83, 78), (33, 87), (68, 70), (55, 56), (48, 7), (70, 62), (85, 5), (26, 46), (64, 89), (30, 90), (99, 25), (60, 46), (36, 37), (72, 32), (86, 18), (37, 94), (66, 86), (24, 60), (77, 10), (47, 93), (26, 14), (86, 6), (78, 2), (98, 89), (11, 60), (91, 10), (18, 82), (21, 72), (0, 70), (81, 32), (21, 78), (26, 75), (86, 30), (74, 80), (54, 24), (31, 90), (14, 63), (35, 64), (19, 5), (19, 48), (82, 37), (98, 68), (1, 14), (84, 28), (20, 58), (55, 79), (61, 75), (64, 38),